In [1]:
import os
import cv2
import torch
import glob as gb
import numpy as np
from torch import nn
from IPython import display
from PIL import Image, ImageDraw
from face_alignment import FaceAlignment, LandmarksType

/home/s439476@ucstaff.win.canberra.edu.au/anaconda3/envs/keira_py37_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#loading image
def Loading():
    samm_path ='../SAMM/Raw_Data/SAMM/*'
    casme_path = '../CASME-II/Cropped/*'
    smic_path = '../SMIC/SMIC_all_cropped/HS/*'
    #get subject
    sub_list = gb.glob(samm_path)
    sub = sub_list[0]
    sub_name = sub.split('/')[-1]
    # get video
    video_list = gb.glob(os.path.join(sub,'*'))
    if len(video_list) > 0:
        video = video_list[0]
    else:
        video = video_list[-1]
    video_name = video.split('/')[-1]
    # get frame
    frame_list = sorted(gb.glob(os.path.join(video,'*.jpg')))
    frame = frame_list[:2]
    return sub_name, video_name, frame_list  

In [3]:
# Define the Procrustes analysis function
def procrustes_analysis(src_pts, dst_pts):
    # Compute the centroid of each set of points
    src_centroid = np.mean(src_pts, axis=0)
    dst_centroid = np.mean(dst_pts, axis=0)

    # Compute the distance of each point from its centroid
    src_dist = np.sqrt(np.sum((src_pts - src_centroid) ** 2, axis=1))
    dst_dist = np.sqrt(np.sum((dst_pts - dst_centroid) ** 2, axis=1))

    # Scale the source points to match the destination points
    src_pts_scaled = src_pts * (dst_dist / src_dist)[:, np.newaxis]

    # Compute the rotation and translation that aligns the scaled source points with the destination points
    U, _, Vt = np.linalg.svd(src_pts_scaled.T.dot(dst_pts))
    R = U.dot(Vt)
    t = dst_centroid - R.dot(src_centroid)

    # Return the transformation matrix
    return np.hstack((R, t[:, np.newaxis]))

In [4]:
def ladmrkloading(frm_dir):
    
    # Load the input image
    img = cv2.imread(frm_dir)

    # Convert the PIL image to a numpy array
    img_np = np.array(img)

    # Convert the numpy array to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Make a prediction with the facial landmark detection model
    landmarks = fa.get_landmarks(gray)

    return gray, landmarks[0]

In [5]:
# def cropFace(img, landmarks):
#     # Detect the facial landmarks
#     landmarks = fa.get_landmarks(img)[0]

#     # Find the bounding box of the face
#     x, y, w, h = cv2.boundingRect(landmarks)

#     # Crop the face from the image
#     face = img[y:y+h, x:x+w]

In [6]:
#Geting location of the first frame
sub_name, video_name, framelist = Loading() 
save_path = 'SAMM_Cropped_Dlib'

# Load the facial landmark detection model from the face_alignment library
fa = FaceAlignment(LandmarksType._2D, device='cpu')

# # Load the reference image
# ref_img = cv2.imread(framelist[0])
ref_img, ref_landmarks = ladmrkloading(framelist[0])
warp_img_size = (ref_img.shape[1],ref_img.shape[0])
# Find the bounding box of the face
x, y, w, h = cv2.boundingRect(ref_landmarks)

In [7]:
#normalize the rest frames:
for frame in framelist:
    #create save path
    frame_name = frame.split('/')[-1]
    frame_save_path = os.path.join(save_path, sub_name, video_name+'_ByLWM')
    folder = os.path.exists(frame_save_path)
    if not folder:
        os.makedirs(frame_save_path)
    frame_save_path = os.path.join(frame_save_path, frame_name)

    # Load the image
    img, landmarks = ladmrkloading(frame)

    # Compute the transformation matrix between the landmarks of the image and the reference image
    M = procrustes_analysis(landmarks, ref_landmarks)
    
    # Apply the transformation matrix to the image to warp it to the same position as the reference image
    warped_img = cv2.warpAffine(img, M[:2, :], warp_img_size)

    # Crop the face from the image
    cropped_face = warped_img[y:y+h, x+10:x+w-10]
    
    # Save the warped image
    cv2.imwrite(frame_save_path, cropped_face)
